Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn
import torch.optim as optim
import sqlite3
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize

#from helper import
from DataPreprocessing import DataPreprocesser

In [ ]:
connection = sqlite3.connect('dota2.db')
cursor = connection.cursor()
MyProcesser = DataPreprocesser(connection, cursor)

Load Data

In [ ]:
DataPreprocesser.to_dataframes(MyProcesser)  # Load the data into dataframes

data = MyProcesser.merge_data()
print(data.head())
X = data.drop(columns=['radiant_win'])
y = data.loc[:, 'radiant_win']

Data Setup

Model Setup

Train Model

Evaluate Model

Performance Analysis

Save Data